In [1]:
import pandas as pd
import asyncio
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()
import re

In [2]:
url = "https://ews.tropmet.res.in/"

In [3]:
await page.goto(url, wait_until='networkidle', timeout=0)
await page.locator("div.slideshow-container").wait_for()

In [4]:
await page.locator("div.slideshow-container strong").all_text_contents()

['Current AQI - 1992025-03-14 07',
 'Forecast AQI - 1922025-03-15 01',
 'Current AQI - 1052024-12-31 23',
 'Forecast AQI - 862025-01-27 10',
 'Current AQI - 1432025-01-22 09',
 'Forecast AQI - 2372025-01-25 10',
 'Current AQI - 1612023-12-31 23',
 'Forecast AQI - 3772025-01-25 10',
 'Current AQI - 1332025-01-23 14',
 'Forecast AQI -432025-01-25 10',
 'Current AQI - 02025-01-23 14',
 'Forecast AQI - 232025-01-25 11',
 'Current AQI -2232025-01-22 09',
 'Forecast AQI -1922025-01-25 10',
 'Current AQI - 842025-01-22 09',
 'Forecast AQI - 862025-01-25 10',
 'Current AQI - 622024-07-11 16',
 'Forecast AQI - 802025-01-25 10']

In [5]:
dates = await page.locator("div.slideshow-container strong").all_text_contents()

In [6]:
dates[2]

'Current AQI - 1052024-12-31 23'

In [7]:
date = [re.search(r'\d{4}-\d{2}-\d{2}', item).group() for item in dates]

In [8]:
date


['2025-03-14',
 '2025-03-15',
 '2024-12-31',
 '2025-01-27',
 '2025-01-22',
 '2025-01-25',
 '2023-12-31',
 '2025-01-25',
 '2025-01-23',
 '2025-01-25',
 '2025-01-23',
 '2025-01-25',
 '2025-01-22',
 '2025-01-25',
 '2025-01-22',
 '2025-01-25',
 '2024-07-11',
 '2025-01-25']

In [9]:
on_date = [elem for index, elem in enumerate(date) if (index + 1) % 2 != 0]
on_date

['2025-03-14',
 '2024-12-31',
 '2025-01-22',
 '2023-12-31',
 '2025-01-23',
 '2025-01-23',
 '2025-01-22',
 '2025-01-22',
 '2024-07-11']

In [10]:
result = await page.locator("text tspan.highcharts-text-outline").all_text_contents()

In [11]:
result

['199\u200b',
 '191\u200b',
 '105\u200b',
 '83\u200b',
 '143\u200b',
 '158\u200b',
 '161\u200b',
 '351\u200b',
 '133\u200b',
 '43\u200b',
 '0\u200b',
 '40\u200b',
 '223\u200b',
 '111\u200b',
 '84\u200b',
 '89\u200b',
 '62\u200b',
 '89\u200b']

In [12]:
All = [element.replace("\u200b", "") for element in result]
All

['199',
 '191',
 '105',
 '83',
 '143',
 '158',
 '161',
 '351',
 '133',
 '43',
 '0',
 '40',
 '223',
 '111',
 '84',
 '89',
 '62',
 '89']

In [13]:
AQI = [elem for index, elem in enumerate(All) if (index + 1) % 2 != 0]
AQI

['199', '105', '143', '161', '133', '0', '223', '84', '62']

In [14]:
city = ['Delhi', 'Pune', 'Mumbai', 'Kolkata', 'Bengalore', 'Lucknow', 'Patna', 'Hyderabad', 'Ahmedabad']

In [15]:
result_date_aqi = [{"AQI": aqi, "on_date": dt, "city": city} for aqi, dt, city in zip(AQI, on_date, city)]
result_date_aqi

[{'AQI': '199', 'on_date': '2025-03-14', 'city': 'Delhi'},
 {'AQI': '105', 'on_date': '2024-12-31', 'city': 'Pune'},
 {'AQI': '143', 'on_date': '2025-01-22', 'city': 'Mumbai'},
 {'AQI': '161', 'on_date': '2023-12-31', 'city': 'Kolkata'},
 {'AQI': '133', 'on_date': '2025-01-23', 'city': 'Bengalore'},
 {'AQI': '0', 'on_date': '2025-01-23', 'city': 'Lucknow'},
 {'AQI': '223', 'on_date': '2025-01-22', 'city': 'Patna'},
 {'AQI': '84', 'on_date': '2025-01-22', 'city': 'Hyderabad'},
 {'AQI': '62', 'on_date': '2024-07-11', 'city': 'Ahmedabad'}]

In [16]:
df = pd.DataFrame(result_date_aqi)

In [17]:
df.to_csv("major_cities_aqi.csv", index=False)

In [18]:
pd.read_csv("major_cities_aqi.csv")

,AQI,on_date,city
0,199,2025-03-14,Delhi
1,105,2024-12-31,Pune
2,143,2025-01-22,Mumbai
3,161,2023-12-31,Kolkata
4,133,2025-01-23,Bengalore
5,0,2025-01-23,Lucknow
6,223,2025-01-22,Patna
7,84,2025-01-22,Hyderabad
8,62,2024-07-11,Ahmedabad
